In [5]:
from pyspark import SparkContext

sc = SparkContext()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-2-556aa194a2f1>:3 

In [6]:
wordsList = ['cat', 'elephant', 'rat' ,'rat', 'cat']
wordsRDD = sc.parallelize(wordsList,4)

In [8]:
def makePlural(word):
    return word + "s"

print(makePlural('cat'))

cats


In [17]:
import hashlib

class TestFailure(Exception):
  pass
class PrivateTestFailure(Exception):
  pass

class Test(object):
  passed = 0
  numTests = 0
  failFast = False
  private = False

  @classmethod
  def setFailFast(cls):
    cls.failFast = True

  @classmethod
  def setPrivateMode(cls):
    cls.private = True

  @classmethod
  def assertTrue(cls, result, msg=""):
    cls.numTests += 1
    if result == True:
      cls.passed += 1
      print("1 test passed.")
    else:
      print("1 test failed. " + msg)
      if cls.failFast:
        if cls.private:
          raise PrivateTestFailure(msg)
        else:
          raise TestFailure(msg)

  @classmethod
  def assertEquals(cls, var, val, msg=""):
    cls.assertTrue(var == val, msg)

  @classmethod
  def assertEqualsHashed(cls, var, hashed_val, msg=""):
    cls.assertEquals(cls._hash(var), hashed_val, msg)

  @classmethod
  def printStats(cls):
    print("{0} / {1} test(s) passed.".format(cls.passed, cls.numTests))

  @classmethod
  def _hash(cls, x):
    return hashlib.sha1(str(x)).hexdigest()

In [18]:
Test.assertEquals(makePlural('rat'), 'rats', 'incorrect reslt: makePlural does not add an s')

1 test passed.


In [23]:
PluralRDD = wordsRDD.map(makePlural)
print(PluralRDD.collect())
Test.assertEquals(PluralRDD.collect(), ['cats', 'elephants', 'rats', 'rats', 'cats'], 'incorrect values for pluralRDD')

['cats', 'elephants', 'rats', 'rats', 'cats']
1 test passed.


In [26]:
PluralLambdaRDD = wordsRDD.map(lambda x: x+'s')
print(PluralLambdaRDD.collect())
Test.assertEquals(PluralRDD.collect(), ['cats','elephants', 'rats', 'rats', 'cats'], 'incorrect values for pluralRDD')

['cats', 'elephants', 'rats', 'rats', 'cats']
1 test passed.


In [30]:
pluralLengths = PluralRDD.map(lambda x: len(x)).collect()
print(pluralLengths)
Test.assertEquals(pluralLengths, [4,9,4,4,4], 'incorrect values for pluralLengths')

[4, 9, 4, 4, 4]
1 test passed.


In [36]:
wordPairs = wordsRDD.map(lambda x:(x,1))
print(wordPairs.collect())
Test.assertEquals(wordPairs.collect(), [('cat', 1), ('elephant',1), ('rat', 1), ('rat',1), ('cat', 1)], 'incorrect value for wordPairs')

[('cat', 1), ('elephant', 1), ('rat', 1), ('rat', 1), ('cat', 1)]
1 test passed.


In [41]:
wordsGrouped = wordPairs.groupByKey()
for key,value in wordsGrouped.collect():
    print('{0}: {1}'.format(key, list(value)))
    
Test.assertEquals(sorted(wordsGrouped.mapValues(lambda x:list(x)).collect()),[('cat', [1,1]), ('elephant', [1]), ('rat', [1,1])], 'incorrect value for wordsGrouped')

cat: [1, 1]
elephant: [1]
rat: [1, 1]
1 test passed.


In [58]:
wordCountsGrouped = wordsGrouped.map(lambda args: (args[0], len(args[1])))
print(wordCountsGrouped.collect())
Test.assertEquals(sorted(wordCountsGrouped.collect()), [('cat',2),('elephant',1), ('rat', 2)], 'incorrect value for wordCountsGrouped')

[('cat', 2), ('elephant', 1), ('rat', 2)]
1 test passed.


In [71]:
wordCounts = wordPairs.reduceByKey(lambda x,y: x+y)
print(wordCounts.collect())

[('cat', 2), ('elephant', 1), ('rat', 2)]


In [76]:
uniqueWords = wordCounts.count()
print(uniqueWords)

Test.assertEquals(uniqueWords, 3, 'incorrect count of uniqueWords')

3
1 test passed.


In [82]:
from operator import add
totalCount = wordCounts.map(lambda args: (args[1])).reduce(add)
average = totalCount / float(uniqueWords)
print(totalCount)
print(round(average,2))

Test.assertEquals(round(average, 2), 1.67, 'incorrect value of average')

5
1.67
1 test passed.


In [188]:
def wordCount(wordListRDD):
    wordPairs = wordsRDD.map(lambda x:(x,1))
    wordCounts = wordPairs.reduceByKey(lambda x,y: x+y)
    
    return wordCounts

print(wordCount(wordsRDD).collect())

Test.assertEquals(sorted(wordCount(wordsRDD).collect()), [('cat',2), ('elephant', 1), ('rat', 2)], 'incorrect definition for wordCount function')

[('cat', 2), ('elephant', 1), ('rat', 2)]
1 test passed.


In [130]:
import re
def removePunctuation(text):
    return re.sub("['!,@#$%^&*()_.:-]", "", text).lower()
    
print(removePunctuation('Hi, you!'))
print(removePunctuation(' No under_score!'))
Test.assertEquals(removePunctuation(" The Elephant's 4 cats. "), ' the elephants 4 cats ', 'incorrect definition for removePunctuation function')

hi you
 no underscore
1 test passed.


In [125]:
!ls

Untitled1.ipynb  Untitled2.ipynb  pg100.txt


In [131]:
shakespeareRDD = (sc.textFile('pg100.txt', 8).map(removePunctuation))
print('\n'.join(shakespeareRDD.zipWithIndex().map(lambda args: '{0}: {1}'.format(args[0], args[1])).take(15)))

the project gutenberg ebook of the complete works of william shakespeare by: 0
william shakespeare: 1
: 2
this ebook is for the use of anyone anywhere at no cost and with: 3
almost no restrictions whatsoever  you may copy it give it away or: 4
reuse it under the terms of the project gutenberg license included: 5
with this ebook or online at wwwgutenbergorg: 6
: 7
 this is a copyrighted project gutenberg ebook details below : 8
     please follow the copyright guidelines in this file     : 9
: 10
title the complete works of william shakespeare: 11
: 12
author william shakespeare: 13
: 14


In [175]:
shakespeareWordsRDD = shakespeareRDD.flatMap(lambda x: x.split(' '))
shakespeareWordCount = shakespeareWordsRDD.count()
print(shakespeareWordsRDD.take(15))
print(shakespeareWordCount)

['the', 'project', 'gutenberg', 'ebook', 'of', 'the', 'complete', 'works', 'of', 'william', 'shakespeare', 'by', 'william', 'shakespeare', '']
1410671


In [181]:
shakeWordsRDD = shakespeareWordsRDD.filter(lambda x:  x != '')
print(shakeWordsRDD.take(15))
shakeWordCount = shakeWordsRDD.count()
print(shakeWordCount)

['the', 'project', 'gutenberg', 'ebook', 'of', 'the', 'complete', 'works', 'of', 'william', 'shakespeare', 'by', 'william', 'shakespeare', 'this']
903972


In [210]:
top15WordsAndCounts = shakeWordsRDD.map(lambda x:(x,1)).reduceByKey(add).takeOrdered(15, lambda args:-args[1])
print('\n'.join(map(lambda args:'{0}: {1}'.format(args[0],args[1]), top15WordsAndCounts)))


the: 27780
and: 26774
i: 20594
to: 18934
of: 18249
a: 14625
you: 13190
my: 12480
that: 10981
in: 10951
is: 9563
not: 8503
for: 8238
with: 8024
it: 7328
